# Feature Selection
**Target:** prepare data for Modelling
Includes 2 stages: 
- Drop string columns made for EDA: super_class and super_service
- Drop service
- Map target columns "class" from "normal - some attack" to "0 - 1"
- Apply OneHotEncoder to protocol_type and flag

### Imports

In [33]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import RobustScaler
import sys
sys.path.append('../')
pd.set_option('display.max_columns',None)
warnings.filterwarnings('ignore')
from src.data_loader import load_test_dataset, load_train_dataset, store_dataset
%matplotlib inline columns

UsageError: unrecognized arguments: columns


### Define function

In [34]:
def preprocess(dataframe, services):
    dataframe.drop(labels=['super_service', 'super_class'], axis="columns", inplace=True)    
    dataframe.loc[dataframe['class'] == "normal", "class"] = 0
    dataframe.loc[dataframe['class'] != 0, "class"] = 1
    dataframe = pd.get_dummies(dataframe, columns = ['protocol_type', 'flag'])
    dataframe = encode_service(dataframe, services)
    return dataframe

### Execute function

In [44]:
def encode_service(df, services):
    encoded_service = pd.get_dummies(df['service'])
    encoded_service = encoded_service.reindex(columns=services, fill_value=0)
    df_encoded = df.drop(columns=['service']).reset_index(drop=True)
    df_encoded = pd.concat([df_encoded, encoded_service], axis=1)
    return df_encoded

In [45]:
def get_unique_services(datasets):
    return np.unique(np.concatenate([df['service'].unique() for df in datasets]))

In [55]:
datasets = [load_train_dataset(purpose='eda'), load_test_dataset(purpose='eda')] 
targets = ['train', 'test']
result = []

services = get_unique_services(datasets)

for df, target in zip(datasets, targets):
    df = preprocess(df, services)
    result.append(df)
    store_dataset(df=df, purpose='modelling', target=target)
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Columns: 124 entries, duration to whois
dtypes: bool(84), float64(15), int64(24), object(1)
memory usage: 48.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 0 to 22542
Columns: 124 entries, duration to whois
dtypes: bool(78), float64(15), int64(30), object(1)
memory usage: 9.6+ MB


Columns in df2 but not in df1: set()
